**<font color = black size=6>实验十:EM算法</font>**

In [562]:
from collections import Counter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import random

**<font color = blue size=4>第一部分:实验任务</font>**

<span style="color:purple">=======  
现在给定了一个班级中所有同学的身高数据文件(height.csv)，但不知道各个学生的性别。假设男生身高服从一高斯分布$N_1(\mu_1,\sigma_1^2)$，女生身高服从另一高斯分布$N_2(\mu_2,\sigma_2^2)$，我们可以写出一个混合高斯模型:$x\sim\alpha_1 N_1(\mu_1,\sigma_1^2)+\alpha_2 N_2(\mu_2,\sigma_2^2)$。请使用EM算法完成对该混合高斯分布的估计(即求出对参数$\alpha_1,\alpha_2,\mu_1,\sigma_1,\mu_2,\sigma_2$的估计值)。我们简化地记$\theta_1=(\alpha_1,\mu_1,\sigma_1)$, $\theta_2=(\alpha_2,\mu_2,\sigma_2)$</span>

<span style="color:purple">该数据集(height.csv)特征信息只包括了1个特征，即学生的身高。我们沿用理论课PPT的设置，将隐变量$z_i$按照男生$z_i=1$、女生$z_i=2$的形式进行描述</span>

**<font color = black size=4>E步（Expectation Step）</font>**

<span style="color:purple">1) 将数据集'height.csv'载入并转换为你需要的格式</span>

In [563]:
D = pd.read_csv('height.csv')
height_dataset = np.array(D) # 将数据集转换为numpy数组的格式

<span style="color:purple">2)初始化  
初始化$t=0$时的参数($\alpha_1(0)$, $\alpha_2(0)$, $\mu_1(0)$, $\mu_2(0)$, $\sigma_1(0)$, $\sigma_2(0)$). </span>

In [564]:
# parameter=[alpha1,alpha2,mu1,mu2,sigma1,sigma2]

# 初始化EM算法的参数
alpha1 = 0.5  # 混合系数_男生
alpha2 = 0.5  # 混合系数_女生
mu1 = 178  # 男生身高均值
mu2 = 160  # 女生身高均值
sigma1 = 5  # 男生身高标准差
sigma2 = 5  # 女生身高标准差

parameter = [alpha1, alpha2, mu1, mu2, sigma1, sigma2]
print(parameter)

[0.5, 0.5, 178, 160, 5, 5]


<span style="color:purple">3)编写函数P(x,parameter,z)  
给定参数$(\alpha_1(t),\alpha_2(t),\mu_1(t),\mu_2(t),\sigma_1(t),\sigma_2(t))$以及数据集D,计算每个样本$x_i$的$P(x_i,z_i|\theta)$.</span>

<span style="color:purple">.  
当$z_i=1$时,$$P(x_i,z_i|\theta)=\alpha_1(t)f_1(x_i|\theta_1(t))$$
    当$z_i=2$时,$$P(x_i,z_i|\theta)=\alpha_2(t)f_2(x_i|\theta_2(t))$$</span>

<span style="color:purple"> .   
其中$f_1(x_i|\theta_1(t))$为样本$x_i$在模型$N_1$中的概率密度,公式如下:
    $$f(x_i|\theta_1(t))=\frac{1}{{\sqrt{2\pi}\sigma_1}} e^{-\frac{(x_i-\mu_1)^2}{2\sigma_1^2}}$$</span>

In [565]:
# parameter=[alpha1,alpha2,mu1,mu2,sigma1,sigma2]

# 高斯分布的概率密度函数
def gaussian_pdf(x, mu, sigma):
    return (1 / (np.sqrt(2 * np.pi) * sigma)) * np.exp(-((x - mu) ** 2) / (2 * sigma ** 2))

def P(x, parameter, z):
    alpha1, alpha2, mu1, mu2, sigma1, sigma2 = parameter

    # 根据z的值计算相应的概率
    if z == 1:
        return alpha1 * gaussian_pdf(x, mu1, sigma1)
    elif z == 2:
        return alpha2 * gaussian_pdf(x, mu2, sigma2)

<span style="color:purple">4)编写函数Y(x,parameter,z)  
给定参数$(\alpha_1(t),\alpha_2(t),\mu_1(t),\mu_2(t),\sigma_1(t),\sigma_2(t))$以及数据集D,计算每个样本$x_i$的$y_{1,i}=P((z_i=1)|x_i,\theta)$和$y_{2,i}=P((z_i=2)|x_i,\theta)$.  
公式如下:  
</span>

$$P((z_i=1)|x_i,\theta) = \frac{\alpha_1(t)f_1(x_i|\theta_1(t))}{\alpha_1(t)f_1(x_i|\theta_1(t))+\alpha_2(t)f_2(x_i|\theta_2(t))}$$  
$$P((z_i=2)|x_i,\theta) = \frac{\alpha_2(t)f_2(x_i|\theta_2(t))}{\alpha_1(t)f_1(x_i|\theta_1(t))+\alpha_2(t)f_2(x_i|\theta_2(t))}$$

In [566]:
# parameter=[alpha1,alpha2,mu1,mu2,sigma1,sigma2]
def Y(x, parameter, z):
    # 计算两个分布的加权概率密度
    P1 = P(x, parameter, 1)
    P2 = P(x, parameter, 2)
    common = P1 + P2
    # 计算后验概率
    y1 = P1 / common
    y2 = P2 / common

    if z == 1:
        return y1
    elif z == 2:
        return y2

<span style="color:purple">5)编写函数Q(x,parameter)  
 计算对数似然函数在该分布和基于$\theta(t)$下的期望值$Q(\theta)$.单个样本的期望值公式如下:$$E_{z_i}logP(x_i,z_i|\theta)=y_{1,i}log(\alpha_1(t)f_1(x_i|\theta_1(t)))+y_{2,i}log(\alpha_2(t)f_2(x_i|\theta_2(t)))$$</span>

In [567]:
# parameter=[alpha1,alpha2,mu1,mu2,sigma1,sigma2]
def Q(x, parameter):
    y1 = Y(x, parameter, 1)
    y2 = Y(x, parameter, 2)
    # 计算两个分布的加权概率密度
    P1 = P(x, parameter, 1)
    P2 = P(x, parameter, 2)
    return y1 * np.log(P1) + y2 * np.log(P2)

**<font color = black size=4>M步 (Maximization Step)</font>**

<span style="color:purple">6)编写函数alpha_expection(D,parameter)  
 给定参数$(\alpha_1(t),\alpha_2(t),\mu_1(t),\mu_2(t),\sigma_1(t),\sigma_2(t))$以及数据集D，计算第$(t+1)$轮的$(\alpha_1(t+1),\alpha_2(t+1))$的更新值.
</span>

$$\alpha_1(t+1)=\frac{\sum_{i=1}^m{y_{1,i}}}{m}$$  
$$\alpha_2(t+1)=\frac{\sum_{i=1}^m{y_{2,i}}}{m}$$

In [568]:
# parameter=[alpha1,alpha2,mu1,mu2,sigma1,sigma2]
def alpha_expection(D, parameter):
    # 计算y1和y2的总和
    sum_y1, sum_y2 = 0, 0
    # 遍历全部样本
    for x in D:
        y1 = Y(x, parameter, 1)
        y2 = Y(x, parameter, 2)
        sum_y1 += y1
        sum_y2 += y2
    # 计算新的alpha值
    m = len(D)
    alpha_next_1 = sum_y1 / m
    alpha_next_2 = sum_y2 / m

    return alpha_next_1, alpha_next_2

<span style="color:purple">7)编写函数mu_expection(D,parameter)  
给定参数$(\alpha_1(t),\alpha_2(t),\mu_1(t),\mu_2(t),\sigma_1(t),\sigma_2(t))$以及数据集D，计算第$(t+1)$轮的$(\mu_1(t+1),\mu_2(t+1))$的更新值.
</span>

$$\mu_1(t+1)=\frac{\sum_{i=1}^m{y_{1,i}x_i}}{\sum_{i=1}^m{y_{1,i}}}$$
$$\mu_2(t+1)=\frac{\sum_{i=1}^m{y_{2,i}x_i}}{\sum_{i=1}^m{y_{2,i}}}$$

In [569]:
# parameter=[alpha1,alpha2,mu1,mu2,sigma1,sigma2]
def mu_expection(D, parameter):
    # 计算y1*x和y2*x的总和
    sum_y1_x, sum_y2_x = 0, 0
    # 计算y1和y2的总和
    sum_y1, sum_y2 = 0, 0
    # 遍历全部样本
    for x in D:
        y1 = Y(x, parameter, 1)
        y2 = Y(x, parameter, 2)
        sum_y1_x += y1 * x
        sum_y2_x += y2 * x
        sum_y1 += y1
        sum_y2 += y2
    # 计算新的mu值
    mu_next_1 = sum_y1_x / sum_y1
    mu_next_2 = sum_y2_x / sum_y2

    return mu_next_1, mu_next_2

<span style="color:purple">8)编写函数sigma_expection(D,parameter,mu_next_1,mu_next_2)  
给定参数$(\alpha_1(t),\alpha_2(t),\mu_1(t),\mu_2(t),\sigma_1(t),\sigma_2(t))$以及数据集D，计算第$(t+1)$轮的$(\sigma_1(t+1),\sigma_2(t+1))$的更新值.
</span>

$$\sigma_1(t+1)=\sqrt{\frac{\sum_{i=1}^m{y_{1,i}(x_i-\mu_1(t+1))^2}}{\sum_{i=1}^m{y_{1,i}}}}$$
$$\sigma_2(t+1)=\sqrt{\frac{\sum_{i=1}^m{y_{2,i}(x_i-\mu_2(t+1))^2}}{\sum_{i=1}^m{y_{2,i}}}}$$

In [570]:
#parameter=[alpha1,alpha2,mu1,mu2,sigma1,sigma2]
def sigma_expection(D, parameter, mu_next_1, mu_next_2):
    # 计算y1*(x-mu_next_1)^2 和 y2*(x-mu_next_2)^2 的总和
    sum_y1_squared_diff, sum_y2_squared_diff = 0, 0
    # 计算y1和y2的总和
    sum_y1, sum_y2 = 0, 0
    # 遍历全部样本
    for x in D:
        y1 = Y(x, parameter, 1)
        y2 = Y(x, parameter, 2)
        sum_y1_squared_diff += y1 * (x - mu_next_1) ** 2
        sum_y2_squared_diff += y2 * (x - mu_next_2) ** 2
        sum_y1 += y1
        sum_y2 += y2

    # 计算新的sigma值
    sigma_next_1 = np.sqrt(sum_y1_squared_diff / sum_y1) 
    sigma_next_2 = np.sqrt(sum_y2_squared_diff / sum_y2) 

    return sigma_next_1, sigma_next_2

**<font color = black size=4>E步和M步的迭代过程</font>**

<span style="color:purple">9) 利用前面编写的函数完成EM算法的迭代过程，直至达到收敛要求。请至少完成【3次】不同的初始值下的迭代过程，并比较选出最好的。  
    收敛要求给出如下几种参考:  
    1.迭代轮数达到指定轮数;  
    2.每轮参数更新的差值小于阈值.</span>

<img src='./EM Algorithm Pseudocode.png'>


我们给出这个数据集的正确相关信息作为参考:$\theta_1:(\alpha_1=0.625,\mu_1=175,\sigma_1=4)$,$\theta_2:(\alpha_2=0.375,\mu_2=165,\sigma_2=6)$

In [571]:
# 定义计算参数更新前后差值的函数
def calculate_param_difference(old_params, new_params):
    diff = 0
    param_ranges = [0.5, 0.5, 175, 175, 5, 5]
    for old, new, range in zip(old_params, new_params, param_ranges):
        diff += abs(new - old) / range
    return diff


# 定义EM算法的迭代过程
def em_algorithm(D, initial_params, max_iterations=200, tolerance=0.001):
    """
    param D: 数据集
    param initial_params: 初始参数
    param max_iterations: 最大迭代次数
    param tolerance: 参数更新的差值阈值
    return: 更新后的参数
    """
    old_params = initial_params
    # 迭代指定轮数
    for _ in range(max_iterations):
        # E步骤: 计算后验概率
        # 这一步已经包含在后面的期望计算中

        # M步骤: 更新参数
        alpha_next_1, alpha_next_2 = alpha_expection(D, old_params)
        mu_next_1, mu_next_2 = mu_expection(D, old_params)
        sigma_next_1, sigma_next_2 = sigma_expection(
            D, old_params, mu_next_1, mu_next_2
        )
        # 更新参数
        new_params = [
            alpha_next_1,
            alpha_next_2,
            mu_next_1,
            mu_next_2,
            sigma_next_1,
            sigma_next_2,
        ]

        # 检查收敛
        param_diff = calculate_param_difference(old_params, new_params)
        # # 参数更新的差值小于阈值 则达到收敛要求 提前结束循环
        if param_diff < tolerance:
            print("参数更新不大,迭代提前终止")
            break
        # 更新参数进行下一次计算
        old_params = new_params
    # 返回最终参数
    return new_params


# 设置不同的初始参数
initial_params_set = [
    parameter,  # 开始设置的参数组合1
    [0.5, 0.5, 175, 165, 5, 5],  # 初始参数组合2
    [0.4, 0.6, 156, 190, 1, 2],  # 初始参数组合3
    [0.6, 0.4, 168, 167, 12, 8],  # 初始参数组合4
    [0.9, 0.4, 188, 144, 2, 2],  # 初始参数组合5
]

# 执行EM算法
results = []
for initial_params in initial_params_set:
    result = em_algorithm(height_dataset, initial_params)
    print(result)
    results.append(result)

# 输出结果
for i, result in enumerate(results):
    print(
        f"初始参数组合{i + 1}结果: θ₁:(α₁={result[0].item():.3f}, μ₁={result[2].item():.1f}, σ₁={result[4].item():.1f}), θ₂:(α₂={result[1].item():.3f}, μ₂={result[3].item():.1f}, σ₂={result[5].item():.1f})"
    )

# 正确的模型参数
correct_params = [0.625, 0.375, 175, 165, 4, 6]
print(
    f"数据集标准参数: θ₁:(α₁={correct_params[0]:.3f}, μ₁={correct_params[2]:.1f}, σ₁={correct_params[4]:.1f}), θ₂:(α₂={correct_params[1]:.3f}, μ₂={correct_params[3]:.1f}, σ₂={correct_params[5]:.1f})"
)
# 利用参数得到的期望结果进行比较
# 比较Q值
Q_values = []
for i, result in enumerate(results):
    Q_value = 0
    for x in height_dataset:
        Q_value += Q(x, result)
    Q_values.append(Q_value)

# 找出Q值最大的初始化参数
max_Q_index = Q_values.index(max(Q_values))
best_initial_params_by_Q = initial_params_set[max_Q_index]
print()
print("利用收敛参数计算得到的期望结果Q进行比较")
print("Q_values:", Q_values)
print(f"最好的初始参数组合为组合{max_Q_index+1}:{best_initial_params_by_Q}")


# 利用和正确的模型参数的差异进行选取
# 比较各个初始化参数得到的结果与正确参数的差异
differences = []
for result in results:
    # 调用计算参数差异的函数进行计算
    diff = calculate_param_difference(correct_params, result)
    differences.append(diff)

# 找出最小差异对应的初始化参数
min_diff_index = differences.index(min(differences))
best_initial_params_by_diff = initial_params_set[min_diff_index]
print()
print("利用同标准模型参数的差距进行选取")
print("param_difference:", differences)
print(f"最好的初始参数组合为组合{min_diff_index+1}:{best_initial_params_by_diff}")

# 标准化Q值和参数差异
Q_values_normalized = Q_values / np.abs(np.mean(Q_values))
differences_normalized = differences / np.abs(np.mean(differences))
# 通过加权平均的方式综合考虑Q值和参数差异
weights = [0.5, 0.5]  # 权重调整
combined_scores = []
for Q_val, diff in zip(Q_values_normalized, differences_normalized):
    score = weights[0] * Q_val - weights[1] * diff  # Q值越大越好，差异越小越好
    combined_scores.append(score)
best_combined_index = combined_scores.index(max(combined_scores))
best_combined_params = initial_params_set[best_combined_index]
print()
print("利用综合指标进行评估")
print("combined_scores:",combined_scores)
print(f"最好的初始参数组合为组合{best_combined_index + 1}: {best_combined_params}")

参数更新不大,迭代提前终止
[array([0.64074885]), array([0.35925115]), array([175.32094459]), array([164.80558518]), array([4.05016916]), array([5.54591965])]
参数更新不大,迭代提前终止
[array([0.61766935]), array([0.38233065]), array([175.44655386]), array([165.23742147]), array([3.99341631]), array([5.72916787])]
参数更新不大,迭代提前终止
[array([0.35915156]), array([0.64084844]), array([164.80369389]), array([175.3203704]), array([5.54511609]), array([4.05043028])]
参数更新不大,迭代提前终止
[array([0.61763856]), array([0.38236144]), array([175.44671166]), array([165.23798864]), array([3.99334492]), array([5.72940785])]
参数更新不大,迭代提前终止
[array([0.64073008]), array([0.35926992]), array([175.32105282]), array([164.80594175]), array([4.05011994]), array([5.54607115])]
初始参数组合1结果: θ₁:(α₁=0.641, μ₁=175.3, σ₁=4.1), θ₂:(α₂=0.359, μ₂=164.8, σ₂=5.5)
初始参数组合2结果: θ₁:(α₁=0.618, μ₁=175.4, σ₁=4.0), θ₂:(α₂=0.382, μ₂=165.2, σ₂=5.7)
初始参数组合3结果: θ₁:(α₁=0.359, μ₁=164.8, σ₁=5.5), θ₂:(α₂=0.641, μ₂=175.3, σ₂=4.1)
初始参数组合4结果: θ₁:(α₁=0.618, μ₁=175.4, σ₁=4.0), θ₂:(

**<font color = blue size=4>第二部分:作业提交</font>**

一、实验课下课前提交完成代码，如果下课前未完成，请将已经完成的部分进行提交，未完成的部分于之后的实验报告中进行补充  
要求:  
1)文件格式为：学号-姓名.ipynb  
2)【不要】提交文件夹、压缩包、数据集等无关文件，只需提交单个ipynb文件即可，如果交错请到讲台前联系助教，删掉之前的错误版本后再进行提交

二、实验报告截止日期： 【11月24日 14:20】
要求：  
1)文件格式为：学号-姓名.pdf  
2)【不要】提交文件夹、压缩包、代码文件、数据集等任何与实验报告无关的文件，只需要提交单个pdf文件即可  
3)文件命名时不需要额外添加“实验几”等额外信息，按照格式提交  
4)每周的实验报告提交地址会变化，且有时间限制，提交时间为下周的实验课开始时，请注意及时提交。

实验十(EM算法)的实验报告上交地址:https://send2me.cn/9UjusmMn/S_Cz3o_FpKQEsA  

三、课堂课件获取地址:https://www.jianguoyun.com/p/DZKTh-IQp5WhChiIn6gFIAA  
实验内容获取地址:https://www.jianguoyun.com/p/DbCHB9wQp5WhChiKn6gFIAA